# VesiCounter

VesiCounter allows for two independent image analysis:

1. Counting round vesicles
2. Counting round vesicles with tagged components based on images from two channels.

# **Data Preparation**

## 1. Mounting Google Drive

We do this to give this notebook access to Google Drive files.

**Action needed:** Follow the instructions.

---



**Instructions**


---


1.   Run below cell
2.   Open newly appeared URL in a new tab
3.   Choose the same Google account as you used to access this notebook 
4.   Click *I allow*
5.   Copy code
6. Return to this notebook an paste it below in the field where it asks for the authorization code
7. Hit Enter
8. You should get: `Mounted at /content/drive/`



---

### Unsure if Google Drive is mounted?

You can always run the cell below to check if Google Drive is mounted. 

If the cell output say: `Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).` proceed to the next cells.






In [ ]:
#@title RUN ME - mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


## 2. Defining inputs/outputs paths

**Action needed:** 
 Run below cell

*Users may also redefine directory names in the cell below. After any edit, run below cell*

---

Runing VesiCounter with `Save: Yes` parameter will overwrite results in `VesiCounter/outputs_only_vesicles_counter` or `outputs_vesicles_components_counter` depending on analysis type therefore if user would like to keep the results we recommend to change name of the directory with results and run below cell.

The same applies if user would like to run VesiCounter solely on new input - instead of adding new images it `VesiCounter/inputs` and run analysis on whole dataset we recommend either to delete all images currently present in `VesiCounter/inputs` or rename it, run below cell and add new input images to newly created `VesiCounter/inputs` directory.


### The directories structure is as follows

```
VesiCounter/
├── inputs/ -> directory with subdirectories for each Z-stack/group of single files
├── outputs_only_vesicles_counter/ -> all images copies, detected vesicles are marked as yellow circles; for visual inspection purposes
├── outputs_vesicles_components_counter/ -> all images copies, detected vesicles/components are marked as yellow circles in both channels
├── outputs_summary/ -> Excel summary files for vesicles counting/vesicles with components counting. The created file is named according to analysis type with timestamp to prevent overwriting results from different run.
```




In [ ]:
#@title RUN ME - define paths

import os

# directory with all inputs
inputs="/content/drive/My Drive/VesiCounter/inputs/" #@param{type:"string"}
# directory with images with vesicles marked
outputs_ves_counter="/content/drive/My Drive/VesiCounter/outputs_only_vesicles_counter/" #@param{type:"string"}
# directory with images with vesicles with components marked
outputs_comp_counter="/content/drive/My Drive/VesiCounter/outputs_vesicles_components_counter/" #@param{type:"string"}
# directory with csv summaries of counting
outputs_summmary="/content/drive/My Drive/VesiCounter/outputs_summary/" #@param{type:"string"}

def create_dirs(dirName):
    try:
        # Create target Directory
        os.mkdir(dirName)
    except FileExistsError:
        pass

for i in [inputs, outputs_ves_counter, outputs_comp_counter, outputs_summmary]:
    create_dirs(i)

## 3. Now upload images to `VesiCounter/inputs/` directory


---
**Attention**

Z-stacks have to be saved to separate files e.g. using Fiji software.



Split Z-stack images must comply with the following naming convention:

**EXPERIMENT NAME.czi - ... C[=0][1/2] ... .format**

e.g.

`Experiment-57 pas-1 1.02.czi - Z=1 C=1.tif`

or

`Experiment-57 pas-1 1.02.czi_z001_c001.tif`

Corresponding images from the same plane from both channels should have different C (1 or 2) in their name so the program can distinguish them. 

Each split Z-stack should be placed in a separate directory. Group of single images can be placed in common directory. All the data directories should be uploaded to the `VesiCounter/inputs/`


**Allowed image formats are png and tif.**

---

If running analysis of counting round vesicles with tagged components, it is essential that all Z-stacks in both channels are loaded, no single image is ommited!



**For the purpose of parameters optimization, please upload only a few, selected Z-stacks/single images.**



## 4. Data preprocessing


**Attention**

After uploading any new images **always run below cell**.


**Action needed:** Run below cell



In [ ]:
#@title RUN ME - import neccessary modules and create sets of corresponding images from different Z-stacks 

import numpy as np
import cv2
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import re
import time
import math
import glob
import re

# Create sets of corresponding images from different Z-stacks

inputs_stacks = {} # 'exp name' = [[], []] for both channels

for dir in glob.glob(inputs+'*/'):

    # Trim spaces from input image names

    %cd -q "$dir"
    ! for f in *; do mv "$f" "${f// /_}" > /dev/null 2>&1; done

    for img in glob.glob(dir+'*'):
        
        exp = img.split('/')[-1]
        exp2 = exp.lower().replace('=', '0')

        if exp == '.DS_store': continue
        
        exp_no = exp.split('.')[0]

        channel = int(re.search('c[0]*[12]{1}', exp2).group(0)[-1]) - 1

        if exp_no not in inputs_stacks.keys():
            inputs_stacks[exp_no] = [[],[], dir]

        inputs_stacks[exp_no][channel].append(exp)

# Sort to get corresponding images in correct order

for key in inputs_stacks:
    for i in range(2):
        inputs_stacks[key][i].sort()


## 5. Custom parametres

**Action needed:** 
Run below cell once

Default parameters may be changed below. 

After any paramater change **do not run this cell again**, use the sliders and the parameters will update dynamically.

---
Analysis parameters will be saved together with quantification results in appropriate csv file in `outputs_summary/` directory.


In [3]:
#@title RUN ME - Set parameters

from ipywidgets import interact
import ipywidgets as widgets

ves_color_param = '                        Set Vesicle Color'

print('                        ' + '#' * len(ves_color_param.strip()))
print(ves_color_param)
print('                        ' + '#' * len(ves_color_param.strip()))
output_slider_variable = widgets.Text()
print('')

def f(vesicle):
    output_slider_variable.value = str(vesicle)

interact(f, vesicle=['red','green', 'blue', 'orange', 'pink']);

print('')

#########################################

ves_channel_param = '                What is the number of vesicle channel?'

print('                ' + '#' * len(ves_channel_param.strip()))
print(ves_channel_param)
print('                ' + '#' * len(ves_channel_param.strip()))
output_slider_channel = widgets.IntText()
print('')

def f_channel(channel):
    output_slider_channel.value = int(channel)

interact(f_channel, channel=['1','2']);

print('')

#########################################

comp_color_param = '                      Set Component Color'

print('                      ' + '#' * len(comp_color_param.strip()))
print(comp_color_param)
print('                      ' + '#' * len(comp_color_param.strip()))
output_slider_variable_comp = widgets.Text()
print('')

def f_comp(component):
    output_slider_variable_comp.value = str(component)

interact(f_comp, component=['green', 'red', 'blue', 'orange', 'pink']);

print('')
print('If not applicable, leave default')
print('')

#########################################

hough = '               Set Hough Circle Transform Parameters'
print('               ' +'#' * len(hough.strip()))
print(hough)
print('               ' +'#' * len(hough.strip()))
print('')

default_params = [25, 15, 12, 15, 30]

#########################################

EX_COUNTER_mindist = widgets.IntSlider()
EX_COUNTER_mindist.value = default_params[0]
print('Circle centers minimal distance; assures that the circles are not too close to each other')
def on_change1(v1):
    default_params[0] = v1['new'] 
EX_COUNTER_mindist.observe(on_change1, names='value')
display(EX_COUNTER_mindist)

EX_COUNTER_p1 = widgets.IntSlider()
EX_COUNTER_p1.value= default_params[1]
print('P1 edge detection parameter; the higher the more strict edge detection algorithm')
def on_change2(v):
    default_params[1] = v['new'] 
EX_COUNTER_p1.observe(on_change2, names='value')
display(EX_COUNTER_p1)

EX_COUNTER_p2 = widgets.IntSlider()
EX_COUNTER_p2.value= default_params[2]
print('P2 center detection parameter; the higher the more strict circle recognition algorithm')
def on_change3(v):
    default_params[2] = v['new'] 
EX_COUNTER_p2.observe(on_change3, names='value')
display(EX_COUNTER_p2)

print('ADVICE: Try lower P1 value and increase P2 value simultaneously')

print('')

EX_COUNTER_minR = widgets.IntSlider()
EX_COUNTER_minR.value= default_params[3]
print('Circle minimum radius; prevents tiny/empty circle detection')
def on_change4(v):
    default_params[3] = v['new'] 
EX_COUNTER_minR.observe(on_change4, names='value')
display(EX_COUNTER_minR)

EX_COUNTER_maxR = widgets.IntSlider()
EX_COUNTER_maxR.value= default_params[4]
print('Circle maximum radius; prevents giant circle detection')
def on_change5(v):
    default_params[4] = v['new'] 
EX_COUNTER_maxR.observe(on_change5, names='value')
display(EX_COUNTER_maxR)

#########################################

save_param = '               Save copy of images with marked circles?'

print('               ' + '#' * len(save_param.strip()))
print(save_param)
print('               ' + '#' * len(save_param.strip()))
save_slider_variable = widgets.Text()
print('')
print(' Save copy of all input images in directory according to analysis type \n (see description of directories structure) with detected circles?')
print('')

def f2(save):
    save_slider_variable.value = str(save)

interact(f2, save=['Yes','No']);
print('')
print(' ADVICE: Recommended only of preliminary image analysis stage for visual inspection \n of detected circles and further parameters adjustments.')
print('')
print(' WARNING: All images from either outputs_vesicles_components_counter/ or outputs_only_vesicles_counter/ \n will be deleted (depending on analysis type)')
print('')

#########################################

colors = '                      Set Colors RGB Parameters'
print('                      ' +'#' * len(colors.strip()))
print(colors)
print('                      ' +'#' * len(colors.strip()))
print('')


black_color = [[0,120], [0,120], [0,120]]
BLACK_R = widgets.IntRangeSlider()
BLACK_R.max = 255
BLACK_R.value= black_color[0]
BLACK_R.description = 'R'
BLACK_G = widgets.IntRangeSlider()
BLACK_G.max = 255
BLACK_G.value= black_color[1]
BLACK_G.description = 'G'
BLACK_B = widgets.IntRangeSlider()
BLACK_B.max = 255
BLACK_B.value= black_color[2]
BLACK_B.description = 'B'

ui_black = widgets.HBox([BLACK_R, BLACK_G, BLACK_B])

print('Black color RGB values range')
def on_change6(v):
    black_color[0] = v['new'] 
BLACK_R.observe(on_change6, names='value')
def on_change7(v):
    black_color[1] = v['new'] 
BLACK_G.observe(on_change7, names='value')
def on_change8(v):
    black_color[2] = v['new'] 
BLACK_B.observe(on_change8, names='value')
display(ui_black)

#########################################

red_color = [[90,255], [0,120], [0,120]]
RED_R = widgets.IntRangeSlider()
RED_R.max = 255
RED_R.value= red_color[0]
RED_R.description = 'R'
RED_G = widgets.IntRangeSlider()
RED_G.max = 255
RED_G.value= red_color[1]
RED_G.description = 'G'
RED_B = widgets.IntRangeSlider()
RED_B.max = 255
RED_B.value= red_color[2]
RED_B.description = 'B'

ui_red = widgets.HBox([RED_R, RED_G, RED_B])

print('Red color RGB values range')
def on_change9(v):
    red_color[0] = v['new'] 
RED_R.observe(on_change9, names='value')
def on_change10(v):
    red_color[1] = v['new'] 
RED_G.observe(on_change10, names='value')
def on_change11(v):
    red_color[2] = v['new'] 
RED_B.observe(on_change11, names='value')
display(ui_red)

#########################################

green_color = [[0,150], [100,255], [0,120]]
GREEN_R = widgets.IntRangeSlider()
GREEN_R.max = 255
GREEN_R.value= green_color[0]
GREEN_R.description = 'R'
GREEN_G = widgets.IntRangeSlider()
GREEN_G.max = 255
GREEN_G.value= green_color[1]
GREEN_G.description = 'G'
GREEN_B = widgets.IntRangeSlider()
GREEN_B.max = 255
GREEN_B.value= green_color[2]
GREEN_B.description = 'B'

ui_green = widgets.HBox([GREEN_R, GREEN_G, GREEN_B])

print('Green color RGB values range')
def on_change12(v):
    green_color[0] = v['new'] 
GREEN_R.observe(on_change12, names='value')
def on_change13(v):
    green_color[1] = v['new'] 
GREEN_G.observe(on_change13, names='value')
def on_change14(v):
    green_color[2] = v['new'] 
GREEN_B.observe(on_change14, names='value')
display(ui_green)
print('')

#########################################

pink_color = [[150,255], [0,100], [100,180]]
PINK_R = widgets.IntRangeSlider()
PINK_R.max = 255
PINK_R.value= pink_color[0]
PINK_R.description = 'R'
PINK_G = widgets.IntRangeSlider()
PINK_G.max = 255
PINK_G.value= pink_color[1]
PINK_G.description = 'G'
PINK_B = widgets.IntRangeSlider()
PINK_B.max = 255
PINK_B.value= pink_color[2]
PINK_B.description = 'B'

ui_pink = widgets.HBox([PINK_R, PINK_G, PINK_B])

print('Pink color RGB values range')
def on_change17(v):
    pink_color[0] = v['new'] 
PINK_R.observe(on_change17, names='value')
def on_change18(v):
    pink_color[1] = v['new'] 
PINK_G.observe(on_change18, names='value')
def on_change19(v):
    pink_color[2] = v['new'] 
PINK_B.observe(on_change19, names='value')
display(ui_pink)
print('')

#########################################

orange_color = [[200,255], [100,180], [0,100]]
ORANGE_R = widgets.IntRangeSlider()
ORANGE_R.max = 255
ORANGE_R.value= orange_color[0]
ORANGE_R.description = 'R'
ORANGE_G = widgets.IntRangeSlider()
ORANGE_G.max = 255
ORANGE_G.value= orange_color[1]
ORANGE_G.description = 'G'
ORANGE_B = widgets.IntRangeSlider()
ORANGE_B.max = 255
ORANGE_B.value= orange_color[2]
ORANGE_B.description = 'B'

ui_orange = widgets.HBox([ORANGE_R, ORANGE_G, ORANGE_B])

print('Orange color RGB values range')
def on_change20(v):
    orange_color[0] = v['new'] 
ORANGE_R.observe(on_change20, names='value')
def on_change21(v):
    orange_color[1] = v['new'] 
ORANGE_G.observe(on_change21, names='value')
def on_change22(v):
    orange_color[2] = v['new'] 
ORANGE_B.observe(on_change22, names='value')
display(ui_orange)
print('')

#########################################

blue_color = [[0,100], [0,100], [100,255]]
BLUE_R = widgets.IntRangeSlider()
BLUE_R.max = 255
BLUE_R.value= blue_color[0]
BLUE_R.description = 'R'
BLUE_G = widgets.IntRangeSlider()
BLUE_G.max = 255
BLUE_G.value= blue_color[1]
BLUE_G.description = 'G'
BLUE_B = widgets.IntRangeSlider()
BLUE_B.max = 255
BLUE_B.value= blue_color[2]
BLUE_B.description = 'B'

ui_blue = widgets.HBox([BLUE_R, BLUE_G, BLUE_B])

print('Blue color RGB values range')
def on_change23(v):
    blue_color[0] = v['new'] 
BLUE_R.observe(on_change23, names='value')
def on_change24(v):
    blue_color[1] = v['new'] 
BLUE_G.observe(on_change24, names='value')
def on_change25(v):
    blue_color[2] = v['new'] 
BLUE_B.observe(on_change25, names='value')
display(ui_blue)
print('')

#########################################

halo_percentage = '             Set Percentage of Black Continuous Halo Around Vesicle'
print('             ' +'#' * len(halo_percentage.strip()))
print(halo_percentage)
print('             ' +'#' * len(halo_percentage.strip()))
print('')

halo_default = 60
HALO = widgets.IntSlider()
HALO.max = 100
HALO.value= halo_default

print(' Minimal percentage of continuous black halo around circle; to detect circles \n at least partially on black background')
def on_change15(v):
    halo_default = v['new'] 
HALO.observe(on_change15, names='value')
display(HALO)
print('')

#########################################

color_perc = '             Set Percentage of Defined Color in Detected Circle'
print('             ' +'#' * len(color_perc.strip()))
print(color_perc)
print('             ' +'#' * len(color_perc.strip()))
print('')


color_percentage_default = 40
COLOR_PERCENTAGE = widgets.IntSlider()
COLOR_PERCENTAGE.max = 100
COLOR_PERCENTAGE.value= color_percentage_default

print(' Minimal percentage of defined color in detected circle; ensures no background \n noise circles are detected')
def on_change16(v):
    color_percentage_default = v['new'] 
COLOR_PERCENTAGE.observe(on_change16, names='value')
display(COLOR_PERCENTAGE)


#########################################

color_perc_comp = "       Set Percentage of Components' Defined Color in Detected Circle"
print('       ' +'#' * len(color_perc_comp.strip()))
print(color_perc_comp)
print('       ' +'#' * len(color_perc_comp.strip()))
print('')


color_percentage_comp_default = 2
COLOR_COMP_PERCENTAGE = widgets.IntSlider()
COLOR_COMP_PERCENTAGE.max = 100
COLOR_COMP_PERCENTAGE.value= color_percentage_comp_default

print(" Minimal percentage of components' defined color in detected circle; ensures no vesicles \n without components are detected")
def on_change26(v):
    color_percentage_comp_default = v['new'] 
COLOR_COMP_PERCENTAGE.observe(on_change26, names='value')
display(COLOR_COMP_PERCENTAGE)



                        #################
                        Set Vesicle Color
                        #################



interactive(children=(Dropdown(description='vesicle', options=('red', 'green', 'blue', 'orange', 'pink'), valu…


                ######################################
                What is the number of vesicle channel?
                ######################################



interactive(children=(Dropdown(description='channel', options=('1', '2'), value='1'), Output()), _dom_classes=…


                      ###################
                      Set Component Color
                      ###################



...

IntSlider(value=60)


             ##################################################
             Set Percentage of Defined Color in Detected Circle
             ##################################################

 Minimal percentage of defined color in detected circle; ensures no background 
 noise circles are detected


IntSlider(value=40)

       ##############################################################
       Set Percentage of Components' Defined Color in Detected Circle
       ##############################################################

 Minimal percentage of components' defined color in detected circle; ensures no vesicles 
 without components are detected


IntSlider(value=2)

## 6. Define helper functions


**Action needed:** Run below cell

---
Description of helper functions:

1. `check_if_color` -> checks if pixel is of color according to RGB values
2. `check_halo` -> checks if halo around found circle (circle radius + 5) has defined percentage of black
3. `check_if_circle_color` -> checks if circle has defined percentage of color inside it


In [ ]:
#@title RUN ME - define helper functions
color_names = {'black':(BLACK_R, BLACK_G, BLACK_B), 'red':(RED_R, RED_G, RED_B), 'green': (GREEN_R, GREEN_G, GREEN_B), 'orange': (ORANGE_R, ORANGE_G, ORANGE_B), 'pink': (PINK_R, PINK_G, PINK_B), 'blue':(BLUE_R, BLUE_G, BLUE_B)}

def check_if_color(rgb, color_name):
    """Check if pixel of defined color"""
    
    color = color_names[color_name]
    if rgb[0] in range(color[0].value[0], color[0].value[1]) and rgb[1] in range(color[1].value[0], color[1].value[1]) and rgb[2] in range(color[2].value[0], color[2].value[1]): 
        return True
    else:
        return False

def check_halo(x,y,r,rgb_exopher):
    """Check if halo around circle radius (r+5) has more than defined percentage of continuous black pixels"""
    
    angle = 0
    halo = []
    halo_color = []

    for point in range(60):
        coor_x = int((r + 5) * math.sin(angle))
        coor_y = int((r + 5) * math.cos(angle))
        halo.append((coor_x +int(x), coor_y +int(y)))
        
        try:
            color = rgb_exopher[coor_y +int(y), coor_x +int(x)]
        except IndexError:
            color = [255,255,255]
        if check_if_color(color, 'black'): 
            halo_color.append('1')
        else:
            halo_color.append('0')
        angle += 6

    tmp = ''.join(halo_color)
    black_contigs = [(match.start(), match.end()) for match in re.finditer('1+', tmp)]

    for el in black_contigs:
        if el[1] - el[0] >= (HALO.value / 100) * len(tmp):
            return True, halo

    return False, []

def check_if_circle_color(x, y, r, rgb_exopher, color_name, vesicle=True):
    """Check if halos inside circle (from 1 to radius+1) have mean of pixels of their defined color greater than defined percentage"""
    
    if vesicle:
        param = COLOR_PERCENTAGE.value
    else:
        param = COLOR_COMP_PERCENTAGE.value

    subcirles_colors = []
    mean = []

    for arch in range(1,int(r)+1):
        angle = 0
        for point in range(60):
            coor_x = int(arch * math.sin(angle))
            coor_y = int(arch * math.cos(angle))
            try:
                color = rgb_exopher[coor_y +int(y), coor_x +int(x)]
            except IndexError:
                angle += 6
                continue

            if check_if_color(color, color_name):
            #if color_functions[output_slider_variable.value](color): 
                subcirles_colors.append(True)
            else:
                subcirles_colors.append(False)
            angle += 6

    mean = round(subcirles_colors.count(True)/len(subcirles_colors) *100,2)
    
    if mean >= param:
        return True
    else:
        return False

# VESICLES COUNTER

**Action needed:** After setting parametres above, run below cell

---

Counts number of vesicles in each Z-stack or single image.

Makes circle Hough transform, filters out circles that do not have defined percentage of their defined color inside them and do not have defined percentage of black continuous halo. Remaining circles are considered as our vesicles.

---

Images with marked vesicles are in `outputs_only_vesicles_counter` directory (if user decided to save them).

Number of vesicles for each single image / Z-stack is in `outputs_summary/vesicles_count.csv`

In [ ]:
#@title RUN ME - detect and count vesicles according to defined parametres 

def find_vesicles_circles(image_path, z_stack_circles):
  
  circles_in_image = []
  img_name = image_path.split('/')[-1]

  # Read original picture for later display
  img_orig = cv2.imread(image_path, cv2.IMREAD_COLOR)

  # Turn original picture to rgb
  rgb_exopher = cv2.cvtColor(img_orig,cv2.COLOR_BGR2RGB)

  # Read and blur picture
  img = cv2.imread(image_path,0)
  img = cv2.medianBlur(img,5)

  # Make Hough Transform on picture
  circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT, 1, EX_COUNTER_mindist.value, 
                             param1=EX_COUNTER_p1.value, param2=EX_COUNTER_p2.value, 
                             minRadius=EX_COUNTER_minR.value, maxRadius=EX_COUNTER_maxR.value)

  #print(circles)
  if circles is not None:
    circles = np.uint16(np.around(circles))

    for i in circles[0,:]:

        save = True
        if len(z_stack_circles) > 0: # if some vesicles were found previously, check if current vesicle is not the same - their centers do not differ more than 10px length
            for center_pair in z_stack_circles:
                d = ((int(center_pair[0]) - int(i[0]))**2) + ((int(center_pair[1]) - int(i[1]))**2)
                if d < 70: # vesicle is in the same position as found on previous photo
                    save = False
                    break

        if len(z_stack_circles) == 0 or save: # no vesicles found yet or new vesicle
            draw, halo = check_halo(i[0], i[1], i[2], rgb_exopher)
            if draw:
                exopher_check = check_if_circle_color(i[0],i[1],i[2], rgb_exopher, output_slider_variable.value)
                if exopher_check:
                    # add info of new vesicle center
                    circles_in_image.append((i[0],i[1]))
                    # draw the outer circle
                    cv2.circle(img_orig,(i[0],i[1]),i[2],(0,255,255),2)
                    # draw the center of the circle
                    cv2.circle(img_orig,(i[0],i[1]),2,(0,0,0),3)

  save = outputs_ves_counter + '{}'.format(img_name)
  if save_slider_variable.value == 'Yes': cv2.imwrite(save, img_orig)

  return circles_in_image

files = glob.glob(outputs_ves_counter+'*')
for f in files:
    os.remove(f)

timestr = time.strftime("%Y%m%d-%H%M%S")
f = open('{}vesicles_count_{}.csv'.format(outputs_summmary, timestr), 'w')
f.write('Experiment' +',' + 'Vesicles_sum' +'\n')

for key in inputs_stacks.keys():
    z_stack_vesicles_circles = []
    for img in inputs_stacks[key][output_slider_channel.value -1]:
        img_path = inputs_stacks[key][-1] + img
        z_stack_vesicles_circles.extend(find_vesicles_circles(img_path, z_stack_vesicles_circles))
    f.write(key +',' + str(len(z_stack_vesicles_circles)) +'\n')


f.write('\n')
f.write('\n')
f.write('######  PARAMETERS  ######\n')
f.write('vesicle color,{}\n'.format(output_slider_variable.value))
f.write('vesicle color ({}) RGB ranges, R: {} - {}, G: {} - {}, B: {} - {}\n'.format(output_slider_variable.value, color_names[output_slider_variable.value][0].value[0], color_names[output_slider_variable.value][0].value[1], color_names[output_slider_variable.value][1].value[0], color_names[output_slider_variable.value][1].value[1], color_names[output_slider_variable.value][2].value[0], color_names[output_slider_variable.value][2].value[1]))
f.write('vesicle color ({}) channel no, {}\n'.format(output_slider_variable.value, output_slider_channel.value ))
f.write('Hough: circle centers minimal distance, {}\n'.format(EX_COUNTER_mindist.value))
f.write('Hough: P1 edge detection parameter, {}\n'.format(EX_COUNTER_p1.value))
f.write('Hough: P2 center detection parameter, {}\n'.format(EX_COUNTER_p2.value))
f.write('Hough: circle minimum radius, {}\n'.format(EX_COUNTER_minR.value))
f.write('Hough: circle maximum radius, {}\n'.format(EX_COUNTER_maxR.value))
f.write('Minimal % of continous black halo around vesicle, {}\n'.format(HALO.value))
f.write('Minimal % of {} color in detected circle, {}\n'.format(output_slider_variable.value, COLOR_PERCENTAGE.value))

f.close()

# VESICLES WITH TAGGED COMPONENTS COUNTER

**Run only if applicable** - if user provided corresponding Z-stacks from both channels.

---

Counts total number of vesicles containing tagged components and percentage of vesicles with tagged components among total vesicles for each Z-stack. 

As previously, makes circle Hough transform, filters out circles that do not have defined percentage of their defined color inside them and do not have defined percentage of black  continuous halo. As remaining circles are considered as our vesicles, check if  their interior contains the defined percentage of pixels of defined color.

For each detected vesicle, checks 3 corresponding images (focal planes -1, 0, +1) from another channel to see whether the vesicle's interior contains the defined percentage of pixels of defined color, e.g. we found red vesicle in the photo 2, we check photos 1, 2, 3 of the green channel to see if the vesicle has some green inside.

---

Images with marked vesicles/components pairs are in `outputs_vesicles_components_counter` directory (if user decided to save them).

Number and percentage of vesicles with components for each experiment is in `outputs_summary/vesicles_components_count.csv`

In [ ]:
#@title RUN ME - detect and count vesicles/components pairs according to defined parametres

second_channel_img = {}

def find_comp_circles(image_path, corr_images_diff_color, z_stack_circles):
  
  circles_in_image = []
  img_name_ex = image_path.split('/')[-1]

  # Read original vesicle-channel picture for later display
  img_orig = cv2.imread(image_path)

  # Turn original vesicle-channel picture to rgb
  rgb_exopher = cv2.cvtColor(img_orig,cv2.COLOR_BGR2RGB)

  # Read and blur vesicle-channel pictures

  img = cv2.imread(image_path,0)
  img = cv2.medianBlur(img,5)

  # Create dictionary of all images from another channel

  for corr_img in corr_images_diff_color:
    img_name_mito = corr_img.split('/')[-1]
    if img_name_mito not in second_channel_img.keys():
        img_mito_orig = cv2.imread(corr_img)
        second_channel_img[img_name_mito] = img_mito_orig

  # Make circle Hough Transform on vesicle-channel picture

  circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT, 1, EX_COUNTER_mindist.value, 
                             param1=EX_COUNTER_p1.value, param2=EX_COUNTER_p2.value, 
                             minRadius=EX_COUNTER_minR.value, maxRadius=EX_COUNTER_maxR.value)


  if circles is not None:
    circles = np.uint16(np.around(circles))
    exopher_with_mito_counter = [0] * len(circles[0])

    for i in circles[0,:]:

        save = True
        if len(z_stack_circles) > 0: # if some vesicles were found previously, check if current vesicles is not the same - their centers do not differ more than 10px length
            for center_pair in z_stack_circles:
                d = ((int(center_pair[0]) - int(i[0]))**2) + ((int(center_pair[1]) - int(i[1]))**2)
                if d < 30: # vesicle is in the same position as found on previous photo
                    save = False
                    break

        if len(z_stack_circles) == 0 or save: # no vesicles found yet or new exopher
            draw, halo = check_halo(i[0], i[1], i[2], rgb_exopher)
            if draw:
                exopher_check = check_if_circle_color(i[0],i[1],i[2], rgb_exopher, output_slider_variable.value)
                
                if exopher_check:
                    # add info of new exopher center
                    circles_in_image.append((i[0],i[1]))

                    for corr_img in corr_images_diff_color: # check for components in corresponding focal plane, +1 and -1 in different channel
                        img_name_mito = corr_img.split('/')[-1]
                        rgb_mito = cv2.cvtColor(second_channel_img[img_name_mito],cv2.COLOR_BGR2RGB)

                        mito_check = check_if_circle_color(i[0],i[1],i[2], rgb_mito, output_slider_variable_comp.value, False)
                        
                        if mito_check:
                            # show vesicle circles with components on vesicle-channel
                            cv2.circle(img_orig,(i[0],i[1]),i[2],(0,255,255),2)
                            # show vesicle circles with components channel
                            cv2.circle(second_channel_img[img_name_mito],(i[0],i[1]),i[2],(0,255,255),2)
            
                            exopher_with_mito_counter[np.where(circles[0] == i)[0][0]] = 1
                            break

  save1 = outputs_comp_counter + '{}'.format(img_name_ex)
  if save_slider_variable.value == 'Yes': 
    cv2.imwrite(save1, img_orig)

  try:
      exopher_with_mito_counter
  except NameError:
      exopher_with_mito_counter = None

  if exopher_with_mito_counter is not None:
    return circles_in_image, sum(exopher_with_mito_counter)
  else:
    return circles_in_image, 0

files = glob.glob(outputs_comp_counter+'*')
for f in files:
    os.remove(f)

timestr = time.strftime("%Y%m%d-%H%M%S")
f = open('{}vesicles_components_count_{}.csv'.format(outputs_summmary, timestr), 'w')
f.write('Experiment' + ',' + 'Vesicles_components_sum' + ',' + 'Vesicles_components_percentage' + '\n')

for key in inputs_stacks.keys():
    dir = inputs_stacks[key][-1]

    if len(inputs_stacks[key][abs(output_slider_channel.value - 2)]) != 0 : # if there are images in another channel
        z_stack_vesicles_circles = []
        all_ves_comp = 0

        for el in range(len(inputs_stacks[key][output_slider_channel.value - 1])):
            img_ves_path = dir + inputs_stacks[key][output_slider_channel.value - 1][el]
            
            if el == 0:
                img_comp_paths = [dir + inputs_stacks[key][abs(output_slider_channel.value - 2)][el], dir + inputs_stacks[key][abs(output_slider_channel.value - 2)][el+1]]
            elif el == len(inputs_stacks[key][output_slider_channel.value -1]) -1:
                img_comp_paths = [dir + inputs_stacks[key][abs(output_slider_channel.value - 2)][el-1], dir + inputs_stacks[key][abs(output_slider_channel.value - 2)][el]]
            else:
                img_comp_paths = [dir + inputs_stacks[key][abs(output_slider_channel.value - 2)][el-1], dir + inputs_stacks[key][abs(output_slider_channel.value - 2)][el], dir + inputs_stacks[key][abs(output_slider_channel.value - 2)][el+1]]
                
            vesicles_circles, vesicles_sum = find_comp_circles(img_ves_path, img_comp_paths, z_stack_vesicles_circles)
            z_stack_vesicles_circles.extend(vesicles_circles)
            all_ves_comp += vesicles_sum

        f.write(key + ',' + str(all_ves_comp) + ',' + str(round(all_ves_comp/len(z_stack_vesicles_circles)*100,2)) + '\n')

f.write('\n')
f.write('\n')
f.write('######  PARAMETERS  ######\n')
f.write('vesicle color,{}\n'.format(output_slider_variable.value))
f.write('vesicle color ({}) RGB ranges, R: {} - {}, G: {} - {}, B: {} - {}\n'.format(output_slider_variable.value, color_names[output_slider_variable.value][0].value[0], color_names[output_slider_variable.value][0].value[1], color_names[output_slider_variable.value][1].value[0], color_names[output_slider_variable.value][1].value[1], color_names[output_slider_variable.value][2].value[0], color_names[output_slider_variable.value][2].value[1]))
f.write('vesicle color ({}) channel no, {}\n'.format(output_slider_variable.value, output_slider_channel.value ))
f.write('components color,{}\n'.format(output_slider_variable_comp.value))
f.write('compartments color ({}) RGB ranges, R: {} - {}, G: {} - {}, B: {} - {}\n'.format(output_slider_variable_comp.value, color_names[output_slider_variable_comp.value][0].value[0], color_names[output_slider_variable_comp.value][0].value[1], color_names[output_slider_variable_comp.value][1].value[0], color_names[output_slider_variable_comp.value][1].value[1], color_names[output_slider_variable_comp.value][2].value[0], color_names[output_slider_variable_comp.value][2].value[1]))
f.write('Hough: circle centers minimal distance, {}\n'.format(EX_COUNTER_mindist.value))
f.write('Hough: P1 edge detection parameter, {}\n'.format(EX_COUNTER_p1.value))
f.write('Hough: P2 center detection parameter, {}\n'.format(EX_COUNTER_p2.value))
f.write('Hough: circle minimum radius, {}\n'.format(EX_COUNTER_minR.value))
f.write('Hough: circle maximum radius, {}\n'.format(EX_COUNTER_maxR.value))
f.write('Minimal % of continous black halo around vesicle, {}\n'.format(HALO.value))
f.write('Minimal % of {} color in detected circle, {}\n'.format(output_slider_variable.value, COLOR_PERCENTAGE.value))
f.write("Minimal % of components' {} color in detected circle, {}\n".format(output_slider_variable_comp.value, COLOR_COMP_PERCENTAGE.value))
f.close()

if save_slider_variable.value == 'Yes': 
    for key in second_channel_img.keys():
        save = outputs_comp_counter + '{}'.format(key)
        cv2.imwrite(save, second_channel_img[key])